In [5]:
import numpy as np  # Import NumPy for efficient array manipulation
import heapq  # Import heapq to implement the priority queue for A* search

# Puzzle configuration
PUZZLE_DIM = 3  # Define the dimension of the puzzle (4x4 for the 15-puzzle)
GOAL_STATE = np.array([i for i in range(1, PUZZLE_DIM**2)] + [0]).reshape((PUZZLE_DIM, PUZZLE_DIM))  
# Create the goal state of the puzzle: a 4x4 grid with numbers 1 to 15 in order and 0 (blank) in the last position.

# Utility functions

def find_blank(state):
    """Finds the position of the blank (0) in the puzzle."""
    # np.argwhere returns the indices of elements in the array that match the condition (blank == 0).
    return tuple(np.argwhere(state == 0)[0])  # Returns the (x, y) coordinates of the blank tile.

def is_goal(state):
    """Checks if the current state is the goal state."""
    return np.array_equal(state, GOAL_STATE)  # Checks if the current state matches the goal state.

def get_possible_moves(state):
    """Returns a list of possible moves based on the blank position (up, down, left, right)."""
    moves = []  # Initialize an empty list to store possible moves
    x, y = find_blank(state)  # Find the current position of the blank tile (0)
    
    # Check if the blank tile can move up (if it's not on the first row)
    if x > 0:  
        moves.append((x - 1, y))  # Add the new position to the list (moving up)
    # Check if the blank tile can move down (if it's not on the last row)
    if x < PUZZLE_DIM - 1:  
        moves.append((x + 1, y))  # Add the new position to the list (moving down)
    # Check if the blank tile can move left (if it's not on the first column)
    if y > 0:  
        moves.append((x, y - 1))  # Add the new position to the list (moving left)
    # Check if the blank tile can move right (if it's not on the last column)
    if y < PUZZLE_DIM - 1:  
        moves.append((x, y + 1))  # Add the new position to the list (moving right)
    
    return moves  # Return the list of valid moves

def move_blank(state, new_blank_pos):
    """Returns a new state after moving the blank to the new position."""
    x, y = find_blank(state)  # Get the current position of the blank
    new_x, new_y = new_blank_pos  # Get the new position for the blank
    new_state = state.copy()  # Make a copy of the current state to avoid modifying the original
    # Swap the blank tile with the tile at the new position
    new_state[x, y], new_state[new_x, new_y] = new_state[new_x, new_y], new_state[x, y]
    return new_state  # Return the new state after the move

def manhattan_distance(state):
    """Calculates the Manhattan Distance heuristic for the current state."""
    distance = 0  # Initialize the total Manhattan distance
    for x in range(PUZZLE_DIM):  # Iterate over each row of the puzzle
        for y in range(PUZZLE_DIM):  # Iterate over each column of the puzzle
            value = state[x, y]  # Get the value of the tile at position (x, y)
            if value != 0:  # Ignore the blank tile (0)
                # Calculate the goal position of the tile (1 to 15) in the goal state
                goal_x, goal_y = divmod(value - 1, PUZZLE_DIM)  
                # Manhattan distance is the sum of the absolute differences in rows and columns
                distance += abs(x - goal_x) + abs(y - goal_y)  
    return distance  # Return the total Manhattan distance

def a_star_search(initial_state):
    """Solves the n²-1 puzzle using the A* search algorithm."""
    # Initialize the frontier (priority queue) with the initial state
    frontier = []
    # Push the initial state onto the frontier, starting with f(n) = 0 (no cost), g(n) = 0 (starting position), and no parent (None)
    heapq.heappush(frontier, (0, tuple(map(tuple, initial_state)), 0, None))  
    
    # Initialize dictionaries to keep track of the cost, states, and parents
    cost_so_far = {tuple(map(tuple, initial_state)): 0}  # Tracks the cost to reach each state
    state_map = {tuple(map(tuple, initial_state)): initial_state.copy()}  # Maps state keys to actual states
    parents = {tuple(map(tuple, initial_state)): None}  # Maps state keys to their parent states

    while frontier:
        # Pop the state with the lowest f(n) from the priority queue (min-heap)
        _, current_key, g_n, parent_key = heapq.heappop(frontier)  
        current_state = state_map[current_key]  # Get the actual state corresponding to the popped key
        
        # If the goal is reached, reconstruct the solution path and return it
        if is_goal(current_state):
            path = []  # Initialize an empty list to store the solution path
            while parent_key:  # Backtrack from the goal to the start using the parent dictionary
                path.append(current_state)  # Add the current state to the path
                current_state = state_map[parent_key]  # Get the parent state
                parent_key = parents[parent_key]  # Get the parent's key
            path.reverse()  # Reverse the path to go from the start to the goal
            return path  # Return the solution path

        # Explore all possible moves (neighbors) from the current state
        for move in get_possible_moves(current_state):
            new_state = move_blank(current_state, move)  # Generate the new state after the move
            new_cost = g_n + 1  # Each move has a uniform cost of 1 (g(n) = g(n-1) + 1)
            
            # Convert the new state to a tuple of tuples to use as a key in dictionaries
            state_key = tuple(map(tuple, new_state))
            
            # If this new state has not been visited or we found a cheaper path to it, update it
            if state_key not in cost_so_far or new_cost < cost_so_far[state_key]:
                cost_so_far[state_key] = new_cost  # Update the cost to reach this state
                f_n = new_cost + manhattan_distance(new_state)  # Calculate f(n) = g(n) + h(n)
                
                # Store the new state in the state_map
                state_map[state_key] = new_state  
                
                # Push the new state onto the frontier with its f(n), cost, and parent state
                heapq.heappush(frontier, (f_n, state_key, new_cost, current_key))  
                
                # Record the parent of this new state (for backtracking)
                parents[state_key] = current_key  

    return None  # If no solution is found, return None

# Initialize puzzle state
initial_state = np.array([i for i in range(1, PUZZLE_DIM**2)] + [0])  # Create a 1D array with numbers 1 to 15 and a 0 (blank)
np.random.shuffle(initial_state)  # Shuffle the array to generate a random initial configuration
initial_state = initial_state.reshape((PUZZLE_DIM, PUZZLE_DIM))  # Reshape the array into a 4x4 grid (2D)

print("Initial State:")
print(initial_state)  # Print the initial scrambled state of the puzzle

# Solve the puzzle using A* search
solution_path = a_star_search(initial_state)  # Call the A* search function to solve the puzzle

# Display the solution path if found
if solution_path:
    print("\nSolution found in {} moves:".format(len(solution_path)))  # Print the number of moves
    for step in solution_path:  # Print each step in the solution path
        print(step, "\n")
else:
    print("No solution found.")  # If no solution was found, print a message


Initial State:
[[6 7 5]
 [8 1 2]
 [0 3 4]]

Solution found in 22 moves:
[[6 7 5]
 [0 1 2]
 [8 3 4]] 

[[6 7 5]
 [1 0 2]
 [8 3 4]] 

[[6 0 5]
 [1 7 2]
 [8 3 4]] 

[[0 6 5]
 [1 7 2]
 [8 3 4]] 

[[1 6 5]
 [0 7 2]
 [8 3 4]] 

[[1 6 5]
 [7 0 2]
 [8 3 4]] 

[[1 0 5]
 [7 6 2]
 [8 3 4]] 

[[1 5 0]
 [7 6 2]
 [8 3 4]] 

[[1 5 2]
 [7 6 0]
 [8 3 4]] 

[[1 5 2]
 [7 0 6]
 [8 3 4]] 

[[1 5 2]
 [7 3 6]
 [8 0 4]] 

[[1 5 2]
 [7 3 6]
 [8 4 0]] 

[[1 5 2]
 [7 3 0]
 [8 4 6]] 

[[1 5 2]
 [7 0 3]
 [8 4 6]] 

[[1 5 2]
 [7 4 3]
 [8 0 6]] 

[[1 5 2]
 [7 4 3]
 [0 8 6]] 

[[1 5 2]
 [0 4 3]
 [7 8 6]] 

[[1 5 2]
 [4 0 3]
 [7 8 6]] 

[[1 0 2]
 [4 5 3]
 [7 8 6]] 

[[1 2 0]
 [4 5 3]
 [7 8 6]] 

[[1 2 3]
 [4 5 0]
 [7 8 6]] 

[[1 2 3]
 [4 5 6]
 [7 8 0]] 

